In [1]:
import numpy as np
import pandas as pd
import glob
import xarray as xr
import matplotlib.pyplot as plt
from scipy import stats
import sys; sys.path.insert(1, '/home/alessandro/scripts_tesi')
import geometry as gm
#import conversion as cnv
#from func_statistics import compute_pdf_nbin, slopes_r_p, slope_time, slope_maps
#from coare_functions import *
import meteo
import lcl
import cartopy.crs as ccrs
crs = ccrs.PlateCarree()
#from plotmap import plot_background
from plotdistr import fb_distribution_npoint, perc_distribution, scatterplot_fit, hist2d
import warnings; warnings.filterwarnings("ignore")

In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# find data
path = '/data/Fabien/GS'
wrf_files_daily = path + '/GS700_WRF_daily_3D.nc'
#fname_list = sorted(glob.glob(wrf_files_daily + '*'))

In [3]:
# open data
ds_tot = xr.open_dataset(wrf_files_daily)        # , concat_dim='Time', combine='nested')  # chunks?

Dimensions of arrays are organized as:
[time, level, y, x]

In [4]:
ds_tot

<xarray.Dataset>
Dimensions:               (y_grid_U: 483, x_grid_U: 976, lev_M: 49, y_grid_V: 484, x_grid_V: 975, y_grid_M: 483, x_grid_M: 975, lev_W: 50, time_counter: 456, axis_nbounds: 2)
Coordinates:
    nav_lat_grid_U        (y_grid_U, x_grid_U) float32 ...
    nav_lon_grid_U        (y_grid_U, x_grid_U) float32 ...
  * lev_M                 (lev_M) float32 1.0 2.0 3.0 4.0 ... 47.0 48.0 49.0
    nav_lat_grid_V        (y_grid_V, x_grid_V) float32 ...
    nav_lon_grid_V        (y_grid_V, x_grid_V) float32 ...
    nav_lat_grid_M        (y_grid_M, x_grid_M) float32 ...
    nav_lon_grid_M        (y_grid_M, x_grid_M) float32 ...
  * lev_W                 (lev_W) float32 1.0 2.0 3.0 4.0 ... 48.0 49.0 50.0
    time_centered         (time_counter) datetime64[ns] ...
  * time_counter          (time_counter) datetime64[ns] 2005-04-01T12:00:00 ....
Dimensions without coordinates: y_grid_U, x_grid_U, y_grid_V, x_grid_V, y_grid_M, x_grid_M, axis_nbounds
Data variables: (12/13)
    time_centered_bounds  (time_counter, axis_nbounds) datetime64[ns] ...
    time_counter_bounds   (time_counter, axis_nbounds) datetime64[ns] ...
    U                     (time_counter, lev_M, y_grid_U, x_grid_U) float32 ...
    V                     (time_counter, lev_M, y_grid_V, x_grid_V) float32 ...
    W                     (time_counter, lev_W, y_grid_M, x_grid_M) float32 ...
    PH                    (time_counter, lev_W, y_grid_M, x_grid_M) float32 ...
    ...                    ...
    T                     (time_counter, lev_M, y_grid_M, x_grid_M) float32 ...
    P                     (time_counter, lev_M, y_grid_M, x_grid_M) float32 ...
    PB                    (lev_M, y_grid_M, x_grid_M) float32 ...
    QVAPOR                (time_counter, lev_M, y_grid_M, x_grid_M) float32 ...
    QCLOUD                (time_counter, lev_M, y_grid_M, x_grid_M) float32 ...
    OLR                   (time_counter, y_grid_M, x_grid_M) float32 ...
Attributes: (12/105)
    name:                            wrfout
    description:                     wrf M grid variables
    title:                           wrf M grid variables
    Conventions:                     CF-1.6
    timeStamp:                       2023-Oct-02 12:39:43 GMT
    uuid:                            cbe9dfe0-2fd6-4bb5-acb0-ac00211ab2a9
    ...                              ...
    ISURBAN:                         1
    ISOILWATER:                      14
    HYBRID_OPT:                      2
    ETAC:                            0.2
    history:                         Wed Nov 29 15:04:29 2023: ncrcat -v U,V,...
    NCO:                             netCDF Operators version 5.0.1 (Homepage...

 float U(Time, bottom_top, south_north, west_east_stag) ;

        U:description = "x-wind component" ;

        U:units = "m s-1" ;

float V(Time, bottom_top, south_north_stag, west_east) ;

        V:description = "y-wind component" ;

        V:units = "m s-1" ;

float W(Time, bottom_top_stag, south_north, west_east) ;

        W:description = "z-wind component" ;

        W:units = "m s-1" ;

float PH(Time, bottom_top_stag, south_north, west_east) ;

        PH:description = "perturbation geopotential" ;

        PH:units = "m2 s-2" ;

float PHB(Time, bottom_top_stag, south_north, west_east) ;

        PHB:description = "base-state geopotential" ;

        PHB:units = "m2 s-2" ;

float T(Time, bottom_top, south_north, west_east) ;

        T:description = "perturbation potential temperature (theta-t0)" ;

        T:units = "K" ;
        
float P(Time, bottom_top, south_north, west_east) ;

        P:description = "perturbation pressure" ;

        P:units = "Pa" ;

float PB(Time, bottom_top, south_north, west_east) ;

        PB:description = "BASE STATE PRESSURE" ;

        PB:units = "Pa" ;
        
 SST ???????????

In [6]:
# grid and geographical coordinates of seemingly good points; 
# format: [j,i] i.e. [lat, lon]
# sure there is a better way to do it

# small rectangle
small_gpoints = [[350,410],[350,265],[65,265],[65,410]]; small_gpoints=np.array(small_gpoints)
small_coords = np.zeros_like(small_gpoints)

for i in range(len(small_gpoints[:,0])):
    small_coords[i,0], small_coords[i,1] = ds_tot.nav_lat_grid_M.sel(y_grid_M=small_gpoints[i,0],x_grid_M=small_gpoints[i,1]), ds_tot.nav_lon_grid_M.sel(y_grid_M=small_gpoints[i,0],x_grid_M=small_gpoints[i,1])

small_coords=np.array(small_coords)


# large rectangle
large_gpoints = [[410,410],[65,410],[65,800],[410,800]]; large_gpoints=np.array(large_gpoints)
large_coords = np.zeros_like(large_gpoints)

for i in range(len(large_gpoints[:,0])):
    large_coords[i,0], large_coords[i,1] = ds_tot.nav_lat_grid_M.sel(y_grid_M=large_gpoints[i,0],x_grid_M=large_gpoints[i,1]), ds_tot.nav_lon_grid_M.sel(y_grid_M=large_gpoints[i,0],x_grid_M=large_gpoints[i,1])

large_coords=np.array(large_coords)


# optional expansion
add_gpoints = [[410,800],[65,800],[65,900],[410,900]]; add_gpoints=np.array(add_gpoints)
add_coords = np.zeros_like(add_gpoints)

for i in range(len(small_gpoints[:,0])):
    add_coords[i,0], add_coords[i,1] = ds_tot.nav_lat_grid_M.sel(y_grid_M=add_gpoints[i,0],x_grid_M=add_gpoints[i,1]), ds_tot.nav_lon_grid_M.sel(y_grid_M=add_gpoints[i,0],x_grid_M=add_gpoints[i,1])

add_coords=np.array(add_coords)

In [7]:
large_coords

array([[ 41, -68],
       [ 35, -68],
       [ 35, -59],
       [ 41, -59]])

In [15]:
# select domain
# starting from the large one

mask_lon = (ds_tot.nav_lon_grid_M >= min(large_coords[:,1])) & (ds_tot.nav_lon_grid_M <= max(large_coords[:,1]))
mask_lat = (ds_tot.nav_lat_grid_M >= min(large_coords[:,0])) & (ds_tot.nav_lat_grid_M <= max(large_coords[:,0]))

In [16]:
type(mask_lon)

xarray.core.dataarray.DataArray

In [17]:
ds = ds_tot.where(mask_lon & mask_lat, drop=True)

MemoryError: Unable to allocate 6.30 PiB for an array with shape (456, 49, 483, 976, 314, 536) and data type float32

In [ ]:
# def lon lat time
lon = ds['XLONG'][0].values
lat = ds['XLAT'][0].values
time = ds['XTIME'].values.astype('<M8[s]')
# land - sea mask
lakemask = ds['LAKEMASK'][0].values
landmask = ds['LANDMASK'][0].values
seamask = 1 - lakemask - landmask
seamask[seamask==0] = np.nan
# def extended land-sea mask
seamask_nocoast = seamask.copy(); seamask_island0 = seamask.copy()
seamask_island0[:int(seamask_island0.shape[0]/2),:] = 1
seamask_island = seamask_island0.copy()
for i in range(0,5):
    for j in range(0,5):
        seamask_nocoast = np.roll(seamask, (i,j), axis=(1,0))*seamask_nocoast
        seamask_island = np.roll(seamask_island0, (-i,-j), axis=(1,0))*seamask_island
seamask_nocoast = seamask_nocoast*seamask_island
seamask = seamask_nocoast.copy()

In [ ]:
# daily varaibles
# select sst, q2, U, PBLH
sst_day = ds['SST'].groupby('XTIME.day').mean().values*seamask
PBLH_day = ds['PBLH'].groupby('XTIME.day').mean().values*seamask

In [ ]:
# read DAILY LCL
LCL_file = '/media/matteo/DAGOBAH/WRF_EUREC4A/CNT/FEB/LCL_day.nc'
ds_lcl = xr.open_dataset(LCL_file) # gia fatto ritaglio lonlat
LCL_day = ds_lcl['LCL'].values*seamask

In [ ]:
# compute anomalies daily data
s = 50
dsst_day = np.zeros(sst_day.shape)
dPBLH_day = np.zeros(PBLH_day.shape)
dLCL_day = np.zeros(LCL_day.shape)
for t in range(0,sst_day.shape[0]):
    if t % 10 == 0:
        print(t)
    # upscale using gaussian filter
    sst_ave = gm.nan_gaussian_filter(sst_day[t],s)*seamask
    dsst_day[t] = sst_day[t] - sst_ave
    pblh_ave = gm.nan_gaussian_filter(PBLH_day[t],s)*seamask
    dPBLH_day[t] = PBLH_day[t] - pblh_ave
    lcl_ave = gm.nan_gaussian_filter(LCL_day[t],s)*seamask
    dLCL_day[t] = LCL_day[t] - lcl_ave

In [ ]:
# DISTRIBUTION PBLH/LCL vs SST bin - DAILY
perc_step = 5
nbins = int(100/perc_step) # Number of bins

In [ ]:
# PERC DISTRIBUTION
x = sst_day.copy()
control = x.reshape(-1)
##### Perc distribution: PBLH
y = PBLH_day[:,:,:].copy()
variable = y.reshape(-1)
pdist_sst, pdist_PBLH, pstd_PBLH, pstd_err_PBLH = perc_distribution(control, variable, nbins, perc_step)
##### Perc distribution: LCL
y = LCL_day[:,:,:].copy()
variable = y.reshape(-1)
pdist_sst, pdist_LCL, pstd_LCL, pstd_err_LCL = perc_distribution(control, variable, nbins, perc_step)

In [ ]:
# FIXED BIN DISTRIBUTION
x = sst_day.copy()
control = x.reshape(-1)
##### Fixed bin distribution: PBLH
y = PBLH_day[:,:,:].copy()
variable = y.reshape(-1)
dist_sst, dist_PBLH, std_err_PBLH, npoint_PBLH = fb_distribution_npoint(control, variable, nbins, perc_step)
##### Fixed bin distribution: LCL
y = LCL_day[:,:,:].copy()
variable = y.reshape(-1)
dist_sst, dist_LCL, std_err_LCL, npoint_LCL = fb_distribution_npoint(control, variable, nbins, perc_step)

In [ ]:
###### Figure
fig = plt.figure(figsize=(6., 6.), constrained_layout=True)
spec = fig.add_gridspec(4, 4)
ax1 = fig.add_subplot(spec[1:, :3]) # 2d distributions.

# ax1.plot(dist_sst-273.15, dist_PBLH, 'o', color='b', mfc='none')
ax1.plot(pdist_sst-273.15, pdist_PBLH, '.',color='b', label='PBLH')
# ax1.plot(dist_sst-273.15, dist_LCL, 'o', color='firebrick', mfc='none')
ax1.plot(pdist_sst-273.15, pdist_LCL, '.',color='firebrick', label='LCL')
ax1.legend()

# ax1.plot(dist_sst-273.15, dist_PBLH-dist_LCL, '.-', color='g', mfc='none')

ax1.set_ylabel('height [m]', fontsize=12); ax1.set_xlabel('SST [˚C]', fontsize=12); ax1.grid()
plt.xlim([ pdist_sst[0]-273.15 - (pdist_sst[1]-pdist_sst[0]),pdist_sst[-1]-273.15 + (pdist_sst[1]-pdist_sst[0]) ])

In [ ]:
# PERC DISTRIBUTION
path_bindist = '/home/matteo/Eureca/COARE35/Results/acquistapace_bindist/perc_dist/daily/'
# np.savetxt(path_bindist + 'pdist_sst_day.txt', pdist_sst)
np.savetxt(path_bindist + 'pdist_PBLH_sst_day.txt', pdist_PBLH)
np.savetxt(path_bindist + 'pdist_LCL_sst_day.txt', pdist_LCL)
np.savetxt(path_bindist + 'pstd_PBLH_sst_day.txt', pstd_PBLH)
np.savetxt(path_bindist + 'pstd_LCL_sst_day.txt', pstd_LCL)

In [ ]:
# FB DISTRIBUTION (no point threshold)
path_bindist = '/home/matteo/Eureca/COARE35/Results/acquistapace_bindist/fb_dist/daily/fullfield/'
np.savetxt(path_bindist + 'dist_sst_day.txt', dist_sst)
np.savetxt(path_bindist + 'dist_PBLH_sst_day.txt', dist_PBLH)
np.savetxt(path_bindist + 'dist_LCL_sst_day.txt', dist_LCL)
np.savetxt(path_bindist + 'std_err_PBLH_sst_day.txt', std_err_PBLH)
np.savetxt(path_bindist + 'std_err_LCL_sst_day.txt', std_err_LCL)
np.savetxt(path_bindist + 'npoint_PBLH_sst_day.txt', npoint_PBLH)
np.savetxt(path_bindist + 'npoint_LCL_sst_day.txt', npoint_LCL)

In [ ]:
# DISTRIBUTION PBLH/LCL vs dSST bin - DAILY
perc_step = 5
nbins = int(100/perc_step) # Number of bins

In [ ]:
# PERC DISTRIBUTION
x = dsst_day.copy()
control = x.reshape(-1)
##### Perc distribution: PBLH
y = PBLH_day[:,:,:].copy()
variable = y.reshape(-1)
pdist_dsst, pdist_PBLH_dsst, pstd_PBLH_dsst, pstd_err_PBLH_dsst = perc_distribution(control, variable, nbins, perc_step)
##### Perc distribution: LCL
y = LCL_day[:,:,:].copy()
variable = y.reshape(-1)
pdist_dsst, pdist_LCL_dsst, pstd_LCL_dsst, pstd_err_LCL_dsst = perc_distribution(control, variable, nbins, perc_step)

In [ ]:
# FIXED BIN DISTRIBUTION
x = dsst_day.copy()
control = x.reshape(-1)
##### Fixed bin distribution: PBLH
y = PBLH_day[:,:,:].copy()
variable = y.reshape(-1)
dist_dsst, dist_PBLH_dsst, std_err_PBLH_dsst, npoint_PBLH_dsst = fb_distribution_npoint(control, variable, nbins, perc_step)
##### Fixed bin distribution: LCL
y = LCL_day[:,:,:].copy()
variable = y.reshape(-1)
dist_dsst, dist_LCL_dsst, std_err_LCL_dsst, npoint_LCL_dsst = fb_distribution_npoint(control, variable, nbins, perc_step)

In [ ]:
###### Figure
fig = plt.figure(figsize=(6., 6.), constrained_layout=True)
spec = fig.add_gridspec(4, 4)
ax1 = fig.add_subplot(spec[1:, :3]) # 2d distributions.

# ax1.plot(dist_dsst, dist_PBLH, 'o', color='b', mfc='none')
ax1.plot(pdist_dsst, pdist_PBLH_dsst, '.',color='b', label='PBLH')
# ax1.plot(dist_dsst, dist_LCL, 'o', color='firebrick', mfc='none')
ax1.plot(pdist_dsst, pdist_LCL_dsst, '.',color='firebrick', label='LCL')
ax1.legend()

# ax1.plot(pdist_dsst, pdist_PBLH_dsst-pdist_LCL_dsst, '.-', color='g', mfc='none')

ax1.set_ylabel('height [m]', fontsize=12); ax1.set_xlabel('SST anomalies [˚C]', fontsize=12); ax1.grid()
plt.xlim([ pdist_dsst[0] - (pdist_dsst[1]-pdist_dsst[0]),pdist_dsst[-1] + (pdist_dsst[1]-pdist_dsst[0]) ])

In [ ]:
# PERC DISTRIBUTION
path_bindist = '/home/matteo/Eureca/COARE35/Results/acquistapace_bindist/perc_dist/daily/'
# np.savetxt(path_bindist + 'pdist_dsst_day.txt', pdist_dsst)
np.savetxt(path_bindist + 'pdist_PBLH_dsst_day.txt', pdist_PBLH_dsst)
np.savetxt(path_bindist + 'pdist_LCL_dsst_day.txt', pdist_LCL_dsst)
np.savetxt(path_bindist + 'pstd_PBLH_dsst_day.txt', pstd_PBLH_dsst)
np.savetxt(path_bindist + 'pstd_LCL_dsst_day.txt', pstd_LCL_dsst)

In [ ]:
# FB DISTRIBUTION (no point threshold)
path_bindist = '/home/matteo/Eureca/COARE35/Results/acquistapace_bindist/fb_dist/daily/fullfield/'
np.savetxt(path_bindist + 'dist_dsst_day.txt', dist_dsst)
np.savetxt(path_bindist + 'dist_PBLH_dsst_day.txt', dist_PBLH_dsst)
np.savetxt(path_bindist + 'dist_LCL_dsst_day.txt', dist_LCL_dsst)
np.savetxt(path_bindist + 'std_err_PBLH_dsst_day.txt', std_err_PBLH_dsst)
np.savetxt(path_bindist + 'std_err_LCL_dsst_day.txt', std_err_LCL_dsst)
np.savetxt(path_bindist + 'npoint_PBLH_dsst_day.txt', npoint_PBLH_dsst)
np.savetxt(path_bindist + 'npoint_LCL_dsst_day.txt', npoint_LCL_dsst)

In [ ]:
# DISTRIBUTION dPBLH/dLCL vs dSST bin - DAILY
perc_step = 5
nbins = int(100/perc_step) # Number of bins

In [ ]:
# PERC DISTRIBUTION
x = dsst_day.copy()
control = x.reshape(-1)
##### Perc distribution: PBLH
y = dPBLH_day[:,:,:].copy()
variable = y.reshape(-1)
pdist_dsst, pdist_dPBLH_dsst, pstd_dPBLH_dsst, pstd_err_dPBLH_dsst = perc_distribution(control, variable, nbins, perc_step)
##### Perc distribution: LCL
y = dLCL_day[:,:,:].copy()
variable = y.reshape(-1)
pdist_dsst, pdist_dLCL_dsst, pstd_dLCL_dsst, pstd_err_dLCL_dsst = perc_distribution(control, variable, nbins, perc_step)

In [ ]:
# FIXED BIN DISTRIBUTION
x = dsst_day.copy()
control = x.reshape(-1)
##### Fixed bin distribution: PBLH
y = dPBLH_day[:,:,:].copy()
variable = y.reshape(-1)
dist_dsst, dist_dPBLH_dsst, std_err_dPBLH_dsst, npoint_dPBLH_dsst = fb_distribution_npoint(control, variable, nbins, perc_step)
##### Fixed bin distribution: LCL
y = dLCL_day[:,:,:].copy()
variable = y.reshape(-1)
dist_dsst, dist_dLCL_dsst, std_err_dLCL_dsst, npoint_dLCL_dsst = fb_distribution_npoint(control, variable, nbins, perc_step)

In [ ]:
###### Figure
fig = plt.figure(figsize=(6., 6.), constrained_layout=True)
spec = fig.add_gridspec(4, 4)
ax1 = fig.add_subplot(spec[1:, :3]) # 2d distributions.

ax1.plot(pdist_dsst, pdist_dPBLH_dsst, '.',color='b', label='PBLH')
ax1.plot(pdist_dsst, pdist_dLCL_dsst, '.',color='firebrick', label='LCL')
ax1.legend()

ax1.set_ylabel('height anomalies [m]', fontsize=12); ax1.set_xlabel('SST anomalies [˚C]', fontsize=12); ax1.grid()
plt.xlim([ pdist_dsst[0] - (pdist_dsst[1]-pdist_dsst[0]),pdist_dsst[-1] + (pdist_dsst[1]-pdist_dsst[0]) ])

In [ ]:
# PERC DISTRIBUTION
path_bindist = '/home/matteo/Eureca/COARE35/Results/acquistapace_bindist/perc_dist/daily/'
# np.savetxt(path_bindist + 'pdist_dsst_day.txt', pdist_dsst)
np.savetxt(path_bindist + 'pdist_dPBLH_dsst_day.txt', pdist_dPBLH_dsst)
np.savetxt(path_bindist + 'pdist_dLCL_dsst_day.txt', pdist_dLCL_dsst)
np.savetxt(path_bindist + 'pstd_dPBLH_dsst_day.txt', pstd_dPBLH_dsst)
np.savetxt(path_bindist + 'pstd_dLCL_dsst_day.txt', pstd_dLCL_dsst)

In [ ]:
# FB DISTRIBUTION (no point threshold)
path_bindist = '/home/matteo/Eureca/COARE35/Results/acquistapace_bindist/fb_dist/daily/fullfield/'
np.savetxt(path_bindist + 'dist_dsst_day.txt', dist_dsst)
np.savetxt(path_bindist + 'dist_dPBLH_dsst_day.txt', dist_dPBLH_dsst)
np.savetxt(path_bindist + 'dist_dLCL_dsst_day.txt', dist_dLCL_dsst)
np.savetxt(path_bindist + 'std_err_dPBLH_dsst_day.txt', std_err_dPBLH_dsst)
np.savetxt(path_bindist + 'std_err_dLCL_dsst_day.txt', std_err_dLCL_dsst)
np.savetxt(path_bindist + 'npoint_dPBLH_dsst_day.txt', npoint_dPBLH_dsst)
np.savetxt(path_bindist + 'npoint_dLCL_dsst_day.txt', npoint_dLCL_dsst)

In [ ]:
# HOURLY DATA

In [ ]:
# hourly varaibles
# select sst, q2, U, PBLH
sst = ds['SST'].values*seamask
PBLH = ds['PBLH'].values*seamask

In [ ]:
# read HOURLY LCL
LCL_file = '/media/fabien/DAGOBAH1/WRF_EUREC4A/CNT/FEB/LCL.nc'
ds_lcl = xr.open_dataset(LCL_file)

In [ ]:
LCL = ds_lcl['LCL'].values*seamask

In [ ]:
# compute anomalies hourly data
s = 50
dsst = np.zeros(sst.shape)
dPBLH = np.zeros(PBLH.shape); dLCL = np.zeros(LCL.shape)
for t in range(0,sst.shape[0]):
    if t % 100 == 0:
        print(t)
    # upscale using gaussian filter
    sst_ave = gm.nan_gaussian_filter(sst[t],s)*seamask
    dsst[t] = sst[t] - sst_ave
    PBLH_ave = gm.nan_gaussian_filter(PBLH[t],s)*seamask
    dPBLH[t] = PBLH[t] - PBLH_ave
    LCL_ave = gm.nan_gaussian_filter(LCL[t],s)*seamask
    dLCL[t] = LCL[t] - LCL_ave

In [ ]:
# DISTRIBUTION dPBLH/dLCL vs dSST bin - HOURLY
thrsld_n = 100
perc_step = 5
nbins = int(100/perc_step) # Number of bins
x = dsst.copy()
control = x.reshape(-1)
##### Perc distribution: dPBLH
y = dPBLH[:,:,:].copy()
variable = y.reshape(-1)
pdist_dsst, pdist_dPBLH, pstd_err_dPBLH = perc_distribution(control, variable, nbins, perc_step)
##### Fixed bin distribution: dPBLH
dist_dsst, dist_dPBLH, std_err_dPBLH, npoint_dPBLH = fb_distribution_mod(control, variable, nbins, thrsld_n, perc_step)
##### Perc distribution: dLCL
y = dLCL[:,:,:].copy()
variable = y.reshape(-1)
pdist_dsst, pdist_dLCL, pstd_err_dLCL = perc_distribution(control, variable, nbins, perc_step)
##### Fixed bin distribution: dLCL
dist_dsst, dist_dLCL, std_err_dLCL, npoint_dLCL = fb_distribution_mod(control, variable, nbins, thrsld_n, perc_step)

In [ ]:
###### Figure
fig = plt.figure(figsize=(6., 6.), constrained_layout=True)
spec = fig.add_gridspec(4, 4)
ax1 = fig.add_subplot(spec[1:, :3]) # 2d distributions.

# ax1.errorbar(dist_dsst, dist_dPBLH, std_err_dPBLH, fmt='o', color='b', mfc='none')
# ax1.errorbar(pdist_dsst, pdist_dPBLH, pstd_err_dPBLH, fmt='.',color='b')
# ax1.errorbar(dist_dsst, dist_dLCL, std_err_dLCL, fmt='o', color='firebrick', mfc='none')
# ax1.errorbar(pdist_dsst, pdist_dLCL, pstd_err_dLCL, fmt='.',color='firebrick')

ax1.plot(dist_dsst, dist_dPBLH, 'o', color='b', mfc='none')
ax1.plot(pdist_dsst, pdist_dPBLH, '.',color='b')
ax1.plot(dist_dsst, dist_dLCL, 'o', color='firebrick', mfc='none')
ax1.plot(pdist_dsst, pdist_dLCL, '.',color='firebrick')

ax1.plot(dist_dsst, dist_dPBLH-dist_dLCL, '.-', color='g', mfc='none')

ax1.set_ylabel('height [m]', fontsize=12); ax1.set_xlabel('SST anomalies [˚C]', fontsize=12); ax1.grid()
plt.xlim([ dist_dsst[0] - + (dist_dsst[1]-dist_dsst[0]),dist_dsst[-1] + (dist_dsst[1]-dist_dsst[0]) ])

In [ ]:
path_bindist = '/home/matteo/Eureca/COARE35/Results/acquistapace_bindist/'
# PERC DISTRIBUTION
np.savetxt(path_bindist + 'pdist_dsst.txt', pdist_dsst)
np.savetxt(path_bindist + 'pdist_dPBLH.txt', pdist_dPBLH)
np.savetxt(path_bindist + 'pdist_dLCL.txt', pdist_dLCL)
np.savetxt(path_bindist + 'pstd_err_dPBLH.txt', pstd_err_dPBLH)
np.savetxt(path_bindist + 'pstd_err_dLCL.txt', pstd_err_dLCL)
# FB DISTRIBUTION (no point threshold)
np.savetxt(path_bindist + 'dist_dsst.txt', dist_dsst)
np.savetxt(path_bindist + 'dist_dPBLH.txt', dist_dPBLH)
np.savetxt(path_bindist + 'dist_dLCL.txt', dist_dLCL)
np.savetxt(path_bindist + 'std_err_dPBLH.txt', std_err_dPBLH)
np.savetxt(path_bindist + 'std_err_dLCL.txt', std_err_dLCL)
np.savetxt(path_bindist + 'npoint_dPBLH.txt', npoint_dPBLH)
np.savetxt(path_bindist + 'npoint_dLCL.txt', npoint_dLCL)

In [ ]:
# corr length
L = 80
nskip = int(L/3)
nt = int(3)
nbin = 50

In [ ]:
# linear regression PBLH vs SST
x = dsst[::nt,::nskip,::nskip].copy()
# y = dPBLH[::nt,::nskip,::nskip].copy()
y = (dPBLH - dLCL)[::nt,::nskip,::nskip].copy()
linreg_res, corr_coeff, p_value = slopes_r_p(x,y)

fig = plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
# scatterplot lin regression 
title = "PBLH' vs SST' - sigma "
xlabel = 'SST anomalies [°C]'; ylabel = "PBLH anomalies [m]"
fig1 = scatterplot_fit(x, y, linreg_res, s, title, xlabel, ylabel, fig)
# histogram 2d 
plt.subplot(1,2,2)
fig2 = hist2d(x, y, s, nbin, title, xlabel, ylabel, fig)

In [ ]:
# PLOT DATA

In [ ]:
##############
# BOXPLOT
##############

In [ ]:
# BOXPLOT
def box_plot(data, labels, edge_color, fill_color, ax):
    medianprops = dict(linestyle='-', linewidth=1, color=edge_color)
    meanpointprops = dict(marker='D', markeredgecolor='black',
                      markerfacecolor=edge_color, markersize=4)
    flierprops = dict(marker='o', markeredgecolor=edge_color, markersize=6,
                  linestyle='none')
    
    bp = ax.boxplot(data, labels=labels, medianprops=medianprops, flierprops=flierprops, \
                    meanprops=meanpointprops, showmeans=True, patch_artist=True)
    
    for element in ['boxes', 'whiskers', 'fliers', 'means', 'medians', 'caps']:
        plt.setp(bp[element], color=edge_color)
    for patch in bp['boxes']:
        patch.set(facecolor=fill_color)       
        
    return bp

def box_plot_pos(data, offset, labels, edge_color, fill_color, ax):
    medianprops = dict(linestyle='-', linewidth=1, color=edge_color)
    meanpointprops = dict(marker='D', markeredgecolor='black',
                      markerfacecolor=edge_color, markersize=4)
    flierprops = dict(marker='o', markeredgecolor=edge_color, markersize=6,
                  linestyle='none')
    
    pos = np.arange(len(labels))+offset
    
    bp = ax.boxplot(data, positions= pos, labels=labels, medianprops=medianprops, flierprops=flierprops, \
                    meanprops=meanpointprops, showmeans=True, patch_artist=True)
    
    for element in ['boxes', 'whiskers', 'fliers', 'means', 'medians', 'caps']:
        plt.setp(bp[element], color=edge_color)
    for patch in bp['boxes']:
        patch.set(facecolor=fill_color)       
        
    return bp

In [ ]:
# build sst perc classes
perc = np.arange(10,100,10)
sst_day_perc = np.zeros(len(perc))
dsst_day_perc = np.zeros(len(perc))

for i in range(0,len(perc)):
    sst_day_perc[i] = np.nanpercentile(sst_day,perc[i])
    dsst_day_perc[i] = np.nanpercentile(dsst_day,perc[i])

In [ ]:
bin_PBLH_sst = []; bin_LCL_sst = []
bin_PBLH_dsst = []; bin_LCL_dsst = []
bin_dPBLH_dsst = []; bin_dLCL_dsst = []
labels = []; dlabels = []
for i in range(0,len(perc)):
    if i == 0:
        pblh_sst = PBLH_day[sst_day < sst_day_perc[i]]; lcl_sst = LCL_day[sst_day < sst_day_perc[i]]
        pblh_dsst = PBLH_day[dsst_day < dsst_day_perc[i]]; lcl_dsst = LCL_day[dsst_day < dsst_day_perc[i]]
        dpblh_dsst = dPBLH_day[dsst_day < dsst_day_perc[i]]; dlcl_dsst = dLCL_day[dsst_day < dsst_day_perc[i]]
    else:
        pblh_sst = PBLH_day[(sst_day >= sst_day_perc[i-1]) & (sst_day < sst_day_perc[i])]
        pblh_dsst = PBLH_day[(dsst_day >= dsst_day_perc[i-1]) & (dsst_day < dsst_day_perc[i])]
        dpblh_dsst = dPBLH_day[(dsst_day >= dsst_day_perc[i-1]) & (dsst_day < dsst_day_perc[i])]
        lcl_sst = LCL_day[(sst_day >= sst_day_perc[i-1]) & (sst_day < sst_day_perc[i])]
        lcl_dsst = LCL_day[(dsst_day >= dsst_day_perc[i-1]) & (dsst_day < dsst_day_perc[i])]
        dlcl_dsst = dLCL_day[(dsst_day >= dsst_day_perc[i-1]) & (dsst_day < dsst_day_perc[i])]
    # create lists
    bin_PBLH_sst.append(pblh_sst); bin_LCL_sst.append(lcl_sst)
    bin_PBLH_dsst.append(pblh_dsst); bin_LCL_dsst.append(lcl_dsst)
    bin_dPBLH_dsst.append(dpblh_dsst); bin_dLCL_dsst.append(dlcl_dsst)
    labels.append(str(np.round(sst_day_perc[i]-273.15,2)))
    dlabels.append(str(np.round(dsst_day_perc[i],2)))
# adding last class, when data > 90perc sst
pblh_sst = PBLH_day[sst_day >= sst_day_perc[-1]]; lcl_sst = LCL_day[sst_day >= sst_day_perc[-1]]
pblh_dsst = PBLH_day[dsst_day >= dsst_day_perc[-1]]; lcl_dsst = LCL_day[dsst_day >= dsst_day_perc[-1]]
dpblh_dsst = dPBLH_day[dsst_day >= dsst_day_perc[-1]]; dlcl_dsst = dLCL_day[dsst_day >= dsst_day_perc[-1]]
bin_PBLH_sst.append(pblh_sst); bin_LCL_sst.append(lcl_sst)
bin_PBLH_dsst.append(pblh_dsst); bin_LCL_dsst.append(lcl_dsst)
bin_dPBLH_dsst.append(dpblh_dsst); bin_dLCL_dsst.append(dlcl_dsst)
labels.append(' '); dlabels.append(' ')       

In [ ]:
# BOXPLOT SOVRAPPOSTO
fig, (ax1, ax2, ax3) = plt.subplots(3, figsize=(10, 12))

bp1 = box_plot(bin_PBLH_sst, labels, 'blue', 'white', ax1)
bp2 = box_plot(bin_LCL_sst, labels, 'firebrick', 'white', ax1)
ax1.legend([bp1["boxes"][0], bp2["boxes"][0]], ['PBLH', 'LCL'])
ax1.set_xlabel('SST [˚C]', fontsize=12); ax1.set_ylabel('height [m]', fontsize=12)

bp1 = box_plot(bin_PBLH_dsst, dlabels, 'blue', 'white', ax2)
bp2 = box_plot(bin_LCL_dsst, dlabels, 'firebrick', 'white', ax2)
ax2.legend([bp1["boxes"][0], bp2["boxes"][0]], ['PBLH', 'LCL'])
ax2.set_xlabel('SST anom [˚C]', fontsize=12); ax2.set_ylabel('height [m]', fontsize=12)

bp1 = box_plot(bin_dPBLH_dsst, dlabels, 'blue', 'white', ax3)
bp2 = box_plot(bin_dLCL_dsst, dlabels, 'firebrick', 'white', ax3)
ax3.legend([bp1["boxes"][0], bp2["boxes"][0]], ['PBLH', 'LCL'])
ax3.set_xlabel('SST anom [˚C]', fontsize=12); ax3.set_ylabel('height anom [m]', fontsize=12)

In [ ]:
# BOXPLOT AFFIANCATO
fig, (ax1, ax2, ax3) = plt.subplots(3, figsize=(10, 12))

offs = 0.25

bp1 = box_plot_pos(bin_PBLH_sst, 0, labels, 'blue', 'white', ax1)
bp2 = box_plot_pos(bin_LCL_sst, offs, labels, 'firebrick', 'white', ax1)
ax1.legend([bp1["boxes"][0], bp2["boxes"][0]], ['PBLH', 'LCL'])
ax1.set_xlabel('SST [˚C]', fontsize=12); ax1.set_ylabel('height [m]', fontsize=12)
ax1.set_xticks(np.arange(len(labels)))

bp1 = box_plot_pos(bin_PBLH_dsst, 0, dlabels, 'blue', 'white', ax2)
bp2 = box_plot_pos(bin_LCL_dsst, offs, dlabels, 'firebrick', 'white', ax2)
ax2.legend([bp1["boxes"][0], bp2["boxes"][0]], ['PBLH', 'LCL'])
ax2.set_xlabel('SST anom [˚C]', fontsize=12); ax2.set_ylabel('height [m]', fontsize=12)
ax2.set_xticks(np.arange(len(labels)))

bp1 = box_plot_pos(bin_dPBLH_dsst, 0, dlabels, 'blue', 'white', ax3)
bp2 = box_plot_pos(bin_dLCL_dsst, offs, dlabels, 'firebrick', 'white', ax3)
ax3.legend([bp1["boxes"][0], bp2["boxes"][0]], ['PBLH', 'LCL'])
ax3.set_xlabel('SST anom [˚C]', fontsize=12); ax3.set_ylabel('height anom [m]', fontsize=12)
ax3.set_xticks(np.arange(len(labels)))

In [ ]:
# HOURLY

In [ ]:
bin_dPBLH = [dPBLH[dsst < -1.5], \
            dPBLH[(dsst >= -1.5) & (dsst < -1)], \
            dPBLH[(dsst >= -1) & (dsst < -0.5)], \
            dPBLH[(dsst >= -0.5) & (dsst < 0)], \
            dPBLH[(dsst >= 0) & (dsst < 0.5)], \
            dPBLH[(dsst >= 0.5) & (dsst < 1)], \
            dPBLH[dsst >= 1]]
bin_dLCL = [dLCL[dsst < -1.5], \
            dLCL[(dsst >= -1.5) & (dsst < -1)], \
            dLCL[(dsst >= -1) & (dsst < -0.5)], \
            dLCL[(dsst >= -0.5) & (dsst < 0)], \
            dLCL[(dsst >= 0) & (dsst < 0.5)], \
            dLCL[(dsst >= 0.5) & (dsst < 1)], \
            dLCL[dsst >= 1]]

dlabels = ['-1.5', '-1', '-0.5', '0', '0.5', '1', '1+']

In [ ]:
fig, ax = plt.subplots()
bp1 = box_plot_pos(bin_dPBLH, 0, dlabels, 'blue', 'white')
bp2 = box_plot_pos(bin_dLCL, 0.2, dlabels, 'firebrick', 'white')
ax.legend([bp1["boxes"][0], bp2["boxes"][0]], ['dPBLH', 'dLCL'])
ax.set_xticks(np.arange(len(labels)))
ax.set_xlabel('SST [˚C]', fontsize=12); ax.set_ylabel('height [m]', fontsize=12)

In [ ]:
# READ DATA

In [ ]:
# DAILY
# PERC DISTRIBUTION
path_bindist = '/home/matteo/Eureca/COARE35/Results/acquistapace_bindist/perc_dist/daily/'
pdist_sst = np.loadtxt(path_bindist + 'pdist_sst.txt')
pdist_PBLH = np.loadtxt(path_bindist + 'pdist_PBLH.txt')
pdist_LCL = np.loadtxt(path_bindist + 'pdist_LCL.txt')
pstd_err_PBLH = np.loadtxt(path_bindist + 'pstd_PBLH.txt')
pstd_err_LCL = np.loadtxt(path_bindist + 'pstd_LCL.txt')
# FB DISTRIBUTION (no point threshold)
path_bindist = '/home/matteo/Eureca/COARE35/Results/acquistapace_bindist/fb_dist/daily/'
dist_sst = np.loadtxt(path_bindist + 'dist_sst.txt')
dist_PBLH = np.loadtxt(path_bindist + 'dist_PBLH.txt')
dist_LCL = np.loadtxt(path_bindist + 'dist_LCL.txt')
std_err_PBLH = np.loadtxt(path_bindist + 'std_err_PBLH.txt')
std_err_LCL = np.loadtxt(path_bindist + 'std_err_LCL.txt')
npoint_PBLH = np.loadtxt(path_bindist + 'npoint_PBLH.txt')
npoint_LCL = np.loadtxt(path_bindist + 'npoint_LCL.txt')

In [ ]:
# HOURLY
# PERC DISTRIBUTION
pdist_dsst = np.loadtxt(path_bindist + 'pdist_dsst.txt')
pdist_dPBLH = np.loadtxt(path_bindist + 'pdist_dPBLH.txt')
pdist_dLCL = np.loadtxt(path_bindist + 'pdist_dLCL.txt')
pstd_err_dPBLH = np.loadtxt(path_bindist + 'pstd_err_dPBLH.txt')
pstd_err_dLCL = np.loadtxt(path_bindist + 'pstd_err_dLCL.txt')
# FB DISTRIBUTION (no point threshold)
dist_dsst = np.loadtxt(path_bindist + 'dist_dsst.txt')
dist_dPBLH = np.loadtxt(path_bindist + 'dist_dPBLH.txt')
dist_dLCL = np.loadtxt(path_bindist + 'dist_dLCL.txt')
std_err_dPBLH = np.loadtxt(path_bindist + 'std_err_dPBLH.txt')
std_err_dLCL = np.loadtxt(path_bindist + 'std_err_dLCL.txt')
npoint_dPBLH = np.loadtxt(path_bindist + 'npoint_dPBLH.txt')
npoint_dLCL = np.loadtxt(path_bindist + 'npoint_dLCL.txt')